In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
!pip install inaSpeechSegmenter
# cai dat thu vien phan loai giong noi

     |████████████████████████████████| 13.9MB 297kB/s 
     |████████████████████████████████| 153kB 40.6MB/s 
     |████████████████████████████████| 81kB 7.2MB/s 
     |████████████████████████████████| 61kB 6.2MB/s 
     |████████████████████████████████| 92kB 7.3MB/s 
     |████████████████████████████████| 163kB 48.1MB/s 
     |████████████████████████████████| 92kB 7.9MB/s 
  Created wheel for sidekit: filename=SIDEKIT-1.3.6.6-cp36-none-any.whl size=186914 sha256=68c500fe31206c9bb3f31855ef9659e355d6f24d40116512d65dd26dd44ef1b1
  Stored in directory: /root/.cache/pip/wheels/85/6f/67/7606ef74d759900d14043cae551e34b3b0a87484ccd1a25a9f
  Created wheel for simplejson: filename=simplejson-3.17.0-cp36-cp36m-linux_x86_64.whl size=114212 sha256=a361671fa10cc38515711bcb65f7a4eecbfbf53bd5919a738bccaf2fdf692c04
  Stored in directory: /root/.cache/pip/wheels/86/c0/83/dcd0339abb2640544bb8e0938aab2d069cef55e5647ce6e097
Successfully built sidekit simplejson
ERROR: chainer 6.5.0 has requirement 

In [3]:
# cai dat ma nguon xu li am thanh
! pip install ffmpeg-python

In [ ]:
#import cac thu vien xu li am thanh
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

In [ ]:
#record am thanh bang mic may tinh
AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}


function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [ ]:
#chay ham record
audio, sr = get_audio()
print(audio)

[  0   0   0 ... -86 -97 -91]


In [ ]:
# chon file am thanh de phan tich
# any media supported by ffmpeg may be used (video, audio, urls)
#media = 'url'
media = 

In [ ]:
# Load the API (Current warning is related to h5py and has no consequences)
from inaSpeechSegmenter import Segmenter, seg2csv

In [ ]:
# create an instance of speech segmenter
# this loads neural networks and may last few seconds
# Warnings have no incidence on the results
seg = Segmenter()

In [ ]:
# segmentation is performed using the __call__ method of the segmenter instance
segmentation = seg(media)

In [ ]:
print(segmentation)